# 1. MPI v Pythonu (`mpi4py`)
`mpi4py` je Python rozhraní nad standardem MPI (Message Passing Interface). Používá se pro paralelní běh více procesů, které si vyměňují zprávy nebo NumPy buffery.

Základní pojmy v této lekci:
- `COMM_WORLD`: výchozí komunikátor (skupina všech spuštěných procesů).
- `rank`: pořadové číslo procesu v komunikátoru.
- `size`: počet procesů v komunikátoru.

Instalace v Linuxu (Debian/Ubuntu):
```bash
sudo apt-get install -y openmpi-bin libopenmpi-dev
python -m pip install mpi4py
```

Na Windows se běžně používá MS-MPI (Microsoft MPI). Po instalaci musí být `mpiexec` dostupný v `PATH` (systémová proměnná s cestami k programům).


## 1.1 Jednoduché `scatter`
Proces s rankem 0 připraví data a `scatter` rozdělí jednu hodnotu každému procesu.

`scatter` je kolektivní operace: všichni členové komunikátoru volají stejnou metodu, ale data rozděluje jen `root` proces.


In [ ]:
%%writefile mpi_example.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    values = [(i + 1) ** 2 for i in range(size)]
else:
    values = None

local_value = comm.scatter(values, root=0)
print(f"Proces {rank}: dostal hodnotu {local_value}")


Spuštění v terminálu:

```bash
mpirun -n 4 python mpi_example.py
```

Význam parametrů:
- `mpirun`: spouštěč MPI procesů,
- `-n 4`: počet spuštěných procesů,
- `python mpi_example.py`: program, který poběží v každém procesu.


In [ ]:
!mpirun -n 4 python mpi_example.py

## 1.2 `Sendrecv` s NumPy polem
V této ukázce každý proces pošle `int32` pole sousedovi a současně přijme pole od předchozího procesu (kruhová topologie).

`Sendrecv` kombinuje odeslání i příjem do jednoho volání. V praxi tím často předejdeme zablokování, které vzniká při nešťastném pořadí samostatných `Send`/`Recv`.


In [ ]:
%%writefile mpi_example2.py
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

send_data = np.full(10, rank, dtype=np.int32)
recv_data = np.empty(10, dtype=np.int32)

source = rank - 1 if rank > 0 else size - 1
destination = (rank + 1) % size

comm.Sendrecv(sendbuf=send_data, dest=destination, recvbuf=recv_data, source=source)

print(f"Proces {rank}: přijal {recv_data} od procesu {source}")


In [ ]:
!mpirun -n 4 python mpi_example2.py